In [36]:
import hashlib

In [9]:
hashlib.sha256(b'UN MURCIELAGO VELOZ').hexdigest()

'dca7db9abbc2dbc5e001c7a890d55ec91366dfd7b02b897fc8a2a0e4744f8d35'

In [10]:
hashlib.sha256(b'UN MURCIELAGO VELOY').hexdigest()

'72e32188f9eeefc57ee94e84071c2867ef4a14538e4fed1579ce89d6ce4112ed'

In [33]:
txs = [b'a',b'b',b'c',b'd']

In [38]:
txshashes = [hashlib.sha256(tx).hexdigest() for tx in txs]

In [39]:
[txshashes[s] for s in [slice(i,i+2) for i in range(0,4,2)]]

[['ca978112ca1bbdcafac231b39a23dc4da786eff8147c4e72b9807785afee48bb',
  '3e23e8160039594a33894f6564e1b1348bbd7a0088d42c4acb73eeaed59c009d'],
 ['2e7d2c03a9507ae265ecf5b5356885a53393a2029d241394997265a1a25aefc6',
  '18ac3e7343f016890c510e93f935261169d9e3f565436429830faf0934f4f8e4']]

In [145]:
from hashlib import sha256


class MerkleNode:
    """
    Stores the hash and the parent.
    """
    def __init__(self, hash):
        self.hash = hash
        self.parent = None
        self.left_child = None
        self.right_child = None
        self.leaves = None


class MerkleTree:
    """
    Stores the leaves and the root hash of the tree.
    """
    def __init__(self, data_chunks):
        leaves = []

        for chunk in data_chunks:
            node = MerkleNode(self.compute_hash(chunk))
            leaves.append(node)

        self.leaves = leaves
        self.root = self.build_merkle_tree(leaves)
        

    def build_merkle_tree(self, leaves):
        """
        Builds the Merkle tree from a list of leaves. In case of an odd number of leaves, the last leaf is duplicated.
        """
        num_leaves = len(leaves)
        if num_leaves == 1:
            return leaves[0]

        parents = []

        i = 0
        while i < num_leaves:
            left_child = leaves[i]
            right_child = leaves[i + 1] if i + 1 < num_leaves else left_child

            parents.append(self.create_parent(left_child, right_child))

            i += 2

        return self.build_merkle_tree(parents)

    def create_parent(self, left_child, right_child):
        parent = MerkleNode(
            self.compute_hash(left_child.hash + right_child.hash))
        
        parent.left_child, parent.right_child = left_child, right_child
        left_child.parent, right_child.parent = parent, parent

        return parent

    @staticmethod
    def compute_hash(data):
        data = data.encode('utf-8')
        return sha256(data).hexdigest()
    
    def get_audit_trail(self, chunk_hash):
        """
        Checks if the leaf exists, and returns the audit trail
        in case it does.
        """
        for leaf in self.leaves:
            if leaf.hash == chunk_hash:
        #        print("Leaf exists")
                return self.generate_audit_trail(leaf)
            
        return False
        
        #return [self.generate_audit_trail(leaf) for leaf in self.leaves if leaf.hash == chunk_hash]

    def generate_audit_trail(self, merkle_node, trail=[]):
        """
        Generates the audit trail in a bottom-up fashion
        """
        if merkle_node == self.root:
            trail.append(merkle_node.hash)
            return trail

        # check if the merkle_node is the left child or the right child
        is_left = merkle_node.parent.left_child == merkle_node
        if is_left:
            # since the current node is left child, right child is
            # needed for the audit trail. We'll need this info later
            # for audit proof.
            trail.append((merkle_node.parent.right_child.hash, not is_left))
            return self.generate_audit_trail(merkle_node.parent, trail)
        
        else:
            trail.append((merkle_node.parent.left_child.hash, is_left))
            return self.generate_audit_trail(merkle_node.parent, trail)


In [146]:
txs = ['a', 'b', 'c', 'd']
merkle_tree = MerkleTree(txs)

In [147]:
print(merkle_tree.root.hash)

58c89d709329eb37285837b042ab6ff72c7c8f74de0446b091b6a0131c102cfd


In [148]:
chunk_hash = MerkleTree.compute_hash("a")

In [149]:
chunk_hash

'ca978112ca1bbdcafac231b39a23dc4da786eff8147c4e72b9807785afee48bb'

In [155]:
audit_trail = merkle_tree.get_audit_trail(chunk_hash)

In [156]:
audit_trail

[('3e23e8160039594a33894f6564e1b1348bbd7a0088d42c4acb73eeaed59c009d', False),
 ('d3a0f1c792ccf7f1708d5422696263e35755a86917ea76ef9242bd4a8cf4891a', False),
 '58c89d709329eb37285837b042ab6ff72c7c8f74de0446b091b6a0131c102cfd',
 ('3e23e8160039594a33894f6564e1b1348bbd7a0088d42c4acb73eeaed59c009d', False),
 ('d3a0f1c792ccf7f1708d5422696263e35755a86917ea76ef9242bd4a8cf4891a', False),
 '58c89d709329eb37285837b042ab6ff72c7c8f74de0446b091b6a0131c102cfd']

In [152]:
def verify_audit_trail(chunk_hash, audit_trail):
    """
    Performs the audit-proof from the audit_trail received
    from the trusted server.
    """
    assert audit_trail is not False
    proof_till_now = chunk_hash
    for node in audit_trail[:-1]:
        hash = node[0]
        is_left = node[1]
        if is_left:
            # the order of hash concatenation depends on whether the
            # the node is a left child or right child of its parent
            proof_till_now = MerkleTree.compute_hash(hash + proof_till_now)
        else:
            proof_till_now = MerkleTree.compute_hash(proof_till_now + hash)
        print(proof_till_now)
    
    # verifying the computed root hash against the actual root hash
    return proof_till_now == audit_trail[-1]

In [153]:
verify_audit_trail(chunk_hash, audit_trail)

62af5c3cb8da3e4f25061e829ebeea5c7513c54949115b1acc225930a90154da
58c89d709329eb37285837b042ab6ff72c7c8f74de0446b091b6a0131c102cfd


True